In [ ]:
!pip install -q pyomo

     |████████████████████████████████| 9.7 MB 5.2 MB/s 
     |████████████████████████████████| 49 kB 4.7 MB/s 


In [ ]:
from pyomo.environ import * 

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

**1. [R] Write a mathematical model to solve the assignment problem explained above. Define all
the variables and constraints clearly. Use appropriate notations and define appropriate sets
to be used in your optimization problem.**

so $x \in {0,1}$    binary domain

General optimization form


F is a matrix in which Quantities are given that must be transported from a factory to another factory every week

L is a matrix in which Unit costs of transportation from a location to another location

**from lawler's linearization**

our objective is

$minimize\sum_{i=1}^{n}\sum_{j=1}^{n}\sum_{k=1}^{n}\sum_{l=1}^{n}F[i,,j]*L[k,l]*X[i,k]*X[j*l]$

after converting into linear programming the new eqation will be:

$minimize\sum_{i=1}^{n}\sum_{j=1}^{n}\sum_{k=1}^{n}\sum_{l=1}^{n}F[i,,j]*L[k,l]*Y[i,k,j,l]$

s.t.

$Y[i,k,j,l]\leq X[i,k]$

$Y[i,k,j,l] \leq X[j,l]$

sum of the two equation will be

$2*Y[i,k,j,l]-X[i,k]-X[j,l] \leq 0$

constraint will be

s.t.

$2*Y[i,k,j,l]-X[i,k]-X[j,l] \leq 0$  for $i,j,k,l \in {1,2,.....n}$       **(no of constraint will be $ n^4)$**

$\sum_{i=1}^{i=n}X[i,j]= 1$ (total factory at jth location which is given equal to 1) $ \\  $ for $j \in {1,2,.....n}$ **(no of constraint will be $ n)$**

$\sum_{j=1}^{j=n}X[i,j]= 1$ (the ith factory at all location which is given equal to 1) $ \\  $ for $i \in {1,2,,.....n}$**(no of constraint will be $ n)$**

$\sum_{i=1}^{n}\sum_{j=1}^{n}\sum_{k=1}^{n}\sum_{l=1}^{n}Y[i,k,j,l]=n^2$ **(no of constraint will be $ 1)$**

$X[i,j]\in$ {0,1} $ \ \ i,j,k,l \in {1,2,.....n}$


**2. [R] If there are n factories and n locations, how many variables and constraints are there in
your model?**

no of variables is 

>>$ n^4+n^2$

no of constraint is 

 >>$ n^4+2*n+1$

In [ ]:
quan_csvfile = pd.read_csv('lab7_ex1_q.csv')
quan_csvfile.rename(columns={'location':'factory'},inplace=True)

quan_csvfile

,factory,1,2,3,4,5,6,7,8
0,1,0,18,22,19,23,20,18,19
1,2,21,0,20,0,19,0,22,20
2,3,22,23,0,0,0,17,16,24
3,4,17,25,0,0,21,22,20,17
4,5,12,19,18,19,0,21,21,23
5,6,20,0,0,17,21,0,20,0
6,7,22,24,28,16,23,0,0,19
7,8,23,29,20,17,24,24,23,0


In [ ]:
row_index_quan=range(8)
column_index_quan=range(8)

In [ ]:
cost_csvfile = pd.read_csv('lab7_ex1_c.csv')


In [ ]:
cost_csvfile.rename(columns={'cost':'location'},inplace=True)

cost_csvfile

,location,1,2,3,4,5,6,7,8
0,1,0,11,13,9,19,20,27,13
1,2,11,0,8,9,11,19,10,15
2,3,13,8,0,18,19,19,27,27
3,4,9,9,18,0,19,20,10,20
4,5,19,11,25,19,0,21,17,26
5,6,20,13,20,20,21,0,28,14
6,7,27,10,17,10,17,28,0,12
7,8,13,15,27,20,19,14,12,0


In [ ]:
row_index_cost=range(8)
column_index_cost=range(8)

In [ ]:
model_lab7_ex1=ConcreteModel()

In [ ]:
# create a model
model_lab7_ex1.x_fl = Var(row_index_cost, column_index_cost, domain= Binary)
model_lab7_ex1.y=Var(row_index_cost, column_index_cost,row_index_quan, column_index_quan,domain=Binary)

In [ ]:
model_lab7_ex1.objective=Objective(expr=sum(quan_csvfile.iloc[i,j+1]*cost_csvfile.iloc[k,l+1]*model_lab7_ex1.y[i,k,j,l] for i in row_index_quan for j in column_index_quan for k in row_index_cost for l in column_index_cost),sense=minimize)

In [ ]:
model_lab7_ex1.constraints = ConstraintList()

In [ ]:
for i in range(8):
  model_lab7_ex1.constraints.add(expr=sum(model_lab7_ex1.x_fl[i,j] for j in range(8)) == 1)

for i in range(8):
  model_lab7_ex1.constraints.add(expr=sum(model_lab7_ex1.x_fl[j,i] for j in range(8)) == 1)



In [ ]:
model_lab7_ex1.constraints.add(expr=sum(model_lab7_ex1.y[i,j,k,l] for i in row_index_quan for j in column_index_quan for k in row_index_cost for l in column_index_cost) == 64)

In [ ]:
for i in row_index_quan:
  for j in column_index_quan:
    for k in row_index_cost:
      for l in column_index_cost:
         model_lab7_ex1.constraints.add(expr=2*model_lab7_ex1.y[i,k,j,l]-model_lab7_ex1.x_fl[i,k]-model_lab7_ex1.x_fl[j,l]<=0)
       



In [ ]:
model_lab7_ex1.pprint()

9 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any : 4113 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 159447 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [ ]:
opt_cbc = SolverFactory('cbc')

In [ ]:
result = opt_cbc.solve(model_lab7_ex1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
print(result.solver.time )

2987.6116490364075


**9. [R] Solve the problem and report which facility must be opened at each location**

factory 1 at location 2

factory 2 at location 8

factory 3 at location 7

factory 4 at location 6

factory 5 at location 1

factory 6 at location 3

factory 7 at location 5

factory 8 at location 4

In [ ]:
# display solution
print('\nObjective = ', model_lab7_ex1.objective())

for i in row_index_cost:
  for j in column_index_cost:
    
    if model_lab7_ex1.x_fl[i,j].value!=0:
      print(f'factory {i+1} at location {j+1}')
      



Objective =  14889.0
factory 1 at location 2
factory 2 at location 8
factory 3 at location 7
factory 4 at location 6
factory 5 at location 1
factory 6 at location 3
factory 7 at location 5
factory 8 at location 4


**change the integer variables in your model to continuous variables**


In [ ]:
# create a model
model_lab7_ex1.X_fl = Var(row_index_cost, column_index_cost, domain= NonNegativeReals)
model_lab7_ex1.Y=Var(row_index_cost, column_index_cost,row_index_quan, column_index_quan,domain=NonNegativeReals)

In [ ]:
model_lab7_ex1.objective.deactivate()

In [ ]:
#deactivate previous constraint
for i in range(1,4114):
  model_lab7_ex1.constraints[i].deactivate()

In [ ]:
model_lab7_ex1.objective=Objective(expr=sum(quan_csvfile.iloc[i,j+1]*cost_csvfile.iloc[k,l+1]*model_lab7_ex1.Y[i,k,j,l] for i in row_index_quan for j in column_index_quan for k in row_index_cost for l in column_index_cost),sense=minimize)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
for i in range(8):
  model_lab7_ex1.constraints.add(expr=sum(model_lab7_ex1.X_fl[i,j] for j in range(8)) == 1)

for i in range(8):
  model_lab7_ex1.constraints.add(expr=sum(model_lab7_ex1.X_fl[j,i] for j in range(8)) == 1)

In [ ]:
model_lab7_ex1.constraints.add(expr=sum(model_lab7_ex1.Y[i,j,k,l] for i in row_index_quan for j in column_index_quan for k in row_index_cost for l in column_index_cost) == 64)

In [ ]:
for i in row_index_quan:
  for j in column_index_quan:
    for k in row_index_cost:
      for l in column_index_cost:
         model_lab7_ex1.constraints.add(expr=2*model_lab7_ex1.Y[i,k,j,l]-model_lab7_ex1.X_fl[j,l]-model_lab7_ex1.X_fl[i,k]<=0)
         
       

In [ ]:
model_lab7_ex1.pprint()

Streaming output truncated to the last 5000 lines.
        (6, 3, 0, 4) :     0 :  None :  None : False :  True : NonNegativeReals
        (6, 3, 0, 5) :     0 :  None :  None : False :  True : NonNegativeReals
        (6, 3, 0, 6) :     0 :  None :  None : False :  True : NonNegativeReals
        (6, 3, 0, 7) :     0 :  None :  None : False :  True : NonNegativeReals
        (6, 3, 1, 0) :     0 :  None :  None : False :  True : NonNegativeReals
        (6, 3, 1, 1) :     0 :  None :  None : False :  True : NonNegativeReals
        (6, 3, 1, 2) :     0 :  None :  None : False :  True : NonNegativeReals
        (6, 3, 1, 3) :     0 :  None :  None : False :  True : NonNegativeReals
        (6, 3, 1, 4) :     0 :  None :  None : False :  True : NonNegativeReals
        (6, 3, 1, 5) :     0 :  None :  None : False :  True : NonNegativeReals
        (6, 3, 1, 6) :     0 :  None :  None : False :  True : NonNegativeReals
        (6, 3, 1, 7) :     0 :  None :  None : False :  True : NonNeg

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
opt_cbc = SolverFactory('cbc')

In [ ]:
result = opt_cbc.solve(model_lab7_ex1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


**10. [R] Now change the integer variables in your model to continuous variables, and re-solve the
problem. Report the solution (only the non-zero values of the solution).**

x[0,1] = 1.0

factory 1 at location 2 

x[1,7] = 1.0

factory 2 at location 8 

x[2,6] = 1.0

factory 3 at location 7 

x[3,5] = 1.0

factory 4 at location 6 

x[4,0] = 1.0

factory 5 at location 1 

x[5,2] = 1.0

factory 6 at location 3 

x[6,4] = 1.0

factory 7 at location 5 

x[7,3] = 1.0

factory 8 at location 4

there are 64 Y variable which are non-zero 


In [ ]:
# display solution
print('\nObjective = ', model_lab7_ex1.objective())

for i in range(8):
  for j in range(8):
    
    if model_lab7_ex1.X_fl[i,j].value!=0:
      
      print(f'factory {i+1} at location {j+1}')
      



Objective =  14889.0
factory 1 at location 2
factory 2 at location 8
factory 3 at location 7
factory 4 at location 6
factory 5 at location 1
factory 6 at location 3
factory 7 at location 5
factory 8 at location 4


In [ ]:
z=0
d={}
for i in range(8):
  for j in range(8):
    for k in range(8):
      for l in range(8):
        if model_lab7_ex1.Y[i,j,k,l].value!=0:
          if model_lab7_ex1.Y[i,j,k,l].value not in d:
            d[model_lab7_ex1.Y[i,j,k,l].value]=1
          else:
            d[model_lab7_ex1.Y[i,j,k,l].value]+=1
          z=z+1
          print(f'y[{i},{j},{k},{l}]')
print(z)
print(d)

y[0,1,0,1]
y[0,1,1,7]
y[0,1,2,6]
y[0,1,3,5]
y[0,1,4,0]
y[0,1,5,2]
y[0,1,6,4]
y[0,1,7,3]
y[1,7,0,1]
y[1,7,1,7]
y[1,7,2,6]
y[1,7,3,5]
y[1,7,4,0]
y[1,7,5,2]
y[1,7,6,4]
y[1,7,7,3]
y[2,6,0,1]
y[2,6,1,7]
y[2,6,2,6]
y[2,6,3,5]
y[2,6,4,0]
y[2,6,5,2]
y[2,6,6,4]
y[2,6,7,3]
y[3,5,0,1]
y[3,5,1,7]
y[3,5,2,6]
y[3,5,3,5]
y[3,5,4,0]
y[3,5,5,2]
y[3,5,6,4]
y[3,5,7,3]
y[4,0,0,1]
y[4,0,1,7]
y[4,0,2,6]
y[4,0,3,5]
y[4,0,4,0]
y[4,0,5,2]
y[4,0,6,4]
y[4,0,7,3]
y[5,2,0,1]
y[5,2,1,7]
y[5,2,2,6]
y[5,2,3,5]
y[5,2,4,0]
y[5,2,5,2]
y[5,2,6,4]
y[5,2,7,3]
y[6,4,0,1]
y[6,4,1,7]
y[6,4,2,6]
y[6,4,3,5]
y[6,4,4,0]
y[6,4,5,2]
y[6,4,6,4]
y[6,4,7,3]
y[7,3,0,1]
y[7,3,1,7]
y[7,3,2,6]
y[7,3,3,5]
y[7,3,4,0]
y[7,3,5,2]
y[7,3,6,4]
y[7,3,7,3]
64
{1.0: 64}


there are 64 Y and 8 X which have non-negative value

**11. [R] Are the optimal costs for both problems same? Are the values of the variables still
integer-valued? If yes, explain why.**

the optimal solution are same 

Yes the value of the X variables are still integer-valued because even after making changes as we are assigning different factories to different locations as each factories could be located at one of the location hence we will still get integer valued variable